# Manipulation.py: Pulling example

This tutorial provides an example of realizing a "pull" motion of a robot arm on a box using of "manipulation.py" and "robot_execution.py". 
In the context of pulling a box with a robot arm, "pulling" refers to the action of applying force to move the box towards or away from a specific direction or location using the robot arm. This typically involves gripping the box securely with the robot's end-effector and then exerting force to shift the box along a desired path or trajectory. 

As always we start with the necessary imports

In [1]:
import numpy as np
import robotic as ry
import manipulation as manip

Now onto a basic configuration with a single blue box

In [2]:
C = ry.Config()
C.addFile(ry.raiPath('../rai-robotModels/scenarios/pandaSingle.g'))

midpoint = np.array([-0.105, 0.4, 0.705-.025+.1])
C.addFrame("box") \
    .setPosition(midpoint) \
    .setShape(ry.ST.ssBox, size=[0.08, 0.12, 0.08, 0.001]) \
    .setColor([1, 0, 0]) \
    .setContact(1) \
    .setMass(.1)

# For convenience, a few definitions:
gripper = "l_gripper"
box = "box"
table = "table"

C.view()

0

Now follows the implementation of the pull() function.  The function orchestrates a sequence of actions using a ManipulationModelling instance M to simulate a pulling operation on an object. Two sub-motions, M1 and M2, are derived from M to handle the pulling and approach phases, respectively. 

In [3]:
def pull(obj: str, target_pos: list[float], info: str="", vis: bool=False) -> tuple:
	M = manip.ManipulationModelling(C, info, ['l_gripper'])
	M.setup_pick_and_place_waypoints(gripper, obj, 1e-1)
	M.pull([1., 2.], obj, gripper, table)
	M.target_xy_position([2.], obj, target_pos)
	M.solve()
	if not M.feasible:
		return False, []

	M1 = M.sub_motion(0, accumulated_collisions=False)
	M1.retractPush([.0, .15], gripper, .03)
	M1.approachPush([.85, 1.], gripper, .03)
	path1 = M1.solve()
	if not M1.feasible:
		return False, []

	M2 = M.sub_motion(1, accumulated_collisions=False)
	path2 = M2.solve()
	if not M2.feasible:
		return False, []

	if vis:
		M1.play(C, 1.)
		C.attach(gripper, obj)
		M2.play(C, 1.)
		C.attach(table, obj)
	
	path = np.append(path1, path2, 0)
	return True, path

In the case of using a real robot, you can extend this function by incorporating the following commented code segment, which utilizes botop to execute the motion paths generated. It's important to note that the pulling action may not function correctly in simulation due to the challenges associated with accurately simulating physics for this type of movement.

In [4]:
bot = ry.BotOp(C, useRealRobot=False)
bot.home(C)

Now we try a random "pull" motion attempt_count number of times. Try to play around with the placePosition variable or different objects if you like

In [5]:
attempt_count = 30
success_count = 0

for l in range(attempt_count):

	target_position = [
		midpoint[0] + np.random.uniform(-.15, .15),
		midpoint[1] + np.random.uniform(-.15, .15),
		0.]
	
	success, path = pull(box, target_position, str(l), vis=True)

	if success:
		bot.move(path, [3])
		while bot.getTimeToEnd() > 0.:
			bot.sync(C)
		
	success_count += 1 if success else 0
	
print(f"Successful motions: {success_count}/{attempt_count}")

  -- feasible:0
     { time: 0.070243, evals: 201, done: 1, feasible: 1, sos: 0.0565112, f: 0, ineq: 0, eq: 0.0466609 }
  -- feasible:sub_motion_0--0
     { time: 0.028233, evals: 40, done: 1, feasible: 1, sos: 0.458356, f: 0, ineq: 5.97413e-08, eq: 0.000153222 }
  -- feasible:sub_motion_1--0
     { time: 0.003643, evals: 8, done: 1, feasible: 1, sos: 0.0712723, f: 0, ineq: 0, eq: 4.3429e-05 }
  -- feasible:1
     { time: 0.039542, evals: 120, done: 1, feasible: 1, sos: 0.0337778, f: 0, ineq: 0, eq: 0.0632293 }
  -- feasible:sub_motion_0--1
     { time: 0.034501, evals: 52, done: 1, feasible: 1, sos: 0.369191, f: 0, ineq: 0.000473421, eq: 0.000267068 }
  -- feasible:sub_motion_1--1
     { time: 0.007352, evals: 7, done: 1, feasible: 1, sos: 0.0161418, f: 0, ineq: 0, eq: 1.01445e-05 }
  -- feasible:2
     { time: 0.029019, evals: 85, done: 1, feasible: 1, sos: 0.030578, f: 0, ineq: 0, eq: 0.0611684 }
  -- feasible:sub_motion_0--2
     { time: 0.01812, evals: 25, done: 1, feasible: 1, so

KeyboardInterrupt: 